In [ ]:
import spikeextractors as se
import hither as hi
import numpy as np
import kachery_p2p as kp
import labbox_ephys as le
from labbox_ephys import sorting_views
from labbox_ephys import recording_views

In [ ]:
def prepare_recording_sorting():
    # Simulate a recording (toy example)
    recording, sorting = se.example_datasets.toy_example(duration=50, num_channels=8, K=5, seed=1)
    R = le.LabboxEphysRecordingExtractor.from_memory(recording, serialize=True, serialize_dtype=np.int16)
    S = le.LabboxEphysSortingExtractor.from_memory(sorting, serialize=True)
    return R, S

In [ ]:
recording, sorting = prepare_recording_sorting()

In [ ]:
sorting_views.AverageWaveformsNew(sorting=sorting, recording=recording)

In [ ]:
sorting_views.ElectrodeGeometry(sorting=sorting, recording=recording)

In [ ]:
sorting_views.Autocorrelograms(sorting=sorting, recording=recording)

In [ ]:
recording_views.Timeseries(recording=recording)